<img src="../common/rfsoc_book_banner.jpg" alt="University of Strathclyde" align="left">

<div class="alert alert-block" style="background-color: #c7b8d6; padding: 10px">
    <p style="color: #222222">
        <b>Note:</b>
        <br>
        This Jupyter notebook uses hardware features of the Zynq UltraScale+ RFSoC device. Therefore, the notebook cells will only execute successfully on an RFSoC platform.
        <br>
        <b>This Jupyter notebook is not compatible with the ZCU216 development board as it does not contain the SD-FEC integrated block.</b>
    </p>
</div>

# Notebook Set H

---

## 04 - FEC Decoding
This is the fourth notebook in the series of five, exploring Soft Decision Forward Error Correction (SD-FEC) on RFSoC. In previous notebooks we have encoded a block of information bits using an integrated SD-FEC block in the RFSoC's programmable logic. The encoded block was then baseband modulated into symbols which resulted in a complex signal. Noise was added to this signal which was then baseband demodulated into soft bits using soft decisions. In this notebook we will supply these soft bits to an SD-FEC integrated block, configured as a decoder, to decode our data and obtain the original information bits.

## Table of Contents
* [1. Introduction](#nb4_introduction)
    * [1.2. Design Overview](#nb4_design_overview)
    * [1.2. Notebook Setup](#nb4_notebook_setup)
* [2. Format LLR Values](#nb4_format_llr)
* [3. Configure Decoder](#nb4_configure_decoder)
* [4. Setup Data Buffers](#nb4_setup_data_buffers)
    * [4.1. Control](#nb4_control)
    * [4.2. Status](#nb4_status)
    * [4.3. Transmit and Receive](#nb4_transmit_and_receive)
* [5. Decode Data](#nb4_decode_data)
    * [5.1. Bit Error Rate](#nb4_ber)
* [6. Conclusion](#nb4_conclusion)

## References
* [1] - [AMD-Xilinx, "Soft-Decision FEC Integrated Block v1.1: LogiCORE IP Product Guide", October 2022](https://docs.xilinx.com/r/en-US/pg256-sdfec-integrated-block)

## Revision
* **v1.0** | 18/01/23 | *First Revision*
* **v1.1** | 19/05/23 | *Minor changes for new development boards (ZCU208 & ZCU216)*

---


## 1. Introduction <a class="anchor" id="nb4_introduction"></a>
This notebook makes use of an SD-FEC block, configured as a decoder in Non-5G New Radio (NR) mode, to decode an encoded data block using the Data-Over-Cable Service Interface Specifications (DOCSIS) standard. The DOCSIS standard supported by the SD-FEC core provides five LDPC codes. The decoder accepts soft bits in the form of Log Likelihood Ratios (LLRs). The formatting of these soft bits into the correct fixed-point value will be demonstrated. After decoding, a Bit Error Rate (BER) calculation will be performed to assess the performance of the LDPC code used. 

### 1.1. Design Overview <a class="anchor" id="nb4_design_overview"></a>
The programmable logic design used in this notebook is illustrated in Figure 1.

<a class="anchor" id="fig-1"></a>
<center><figure>
<img src='./images/sdfec_decoder_block_design.svg' width='1000'/>
    <figcaption><b>Figure 1: Functional block diagram illustrating the loop-back implementation of the SD-FEC decoder.</b></figcaption>
</figure></center>

An SD-FEC block is configured as an LDPC decoder and is in a loop-back architecture with the processing system. Four buffers are required for the movement of data between our Jupyter environment and the SD-FEC core. Two DMAs (ctrl and data) are employed to move data between the Processing System (PS) and Programmable Logic (PL). The ctrl DMA is responsible for the control and status buffers and the data DMA is responsible for the transmit and receive buffers. The transmit buffer will contain formatted LLR values or soft bits. This buffer will be sent to the SD-FEC block where the data will be decoded. The decoded block will be received in the receive buffer and we can compare this to the original information bits that were generated in notebook 2.

### 1.2. Notebook Setup <a class="anchor" id="nb4_notebook_setup"></a>
We must first setup the notebook by importing the required libraries and downloading the bitstream to the board. We will also import the soft bits (*llrs*) from notebook 3 for decoding and the original information bits (*tx_enc_buf*) from notebook 2 for comparison.

In [ ]:
from pynq import allocate
import numpy as np
import xsdfec
import strath_sdfec.helper_functions as hf

from strath_sdfec.overlay import SdfecOverlay
ol = SdfecOverlay()

%store -r llrs tx_enc_buf

## 2. Format LLR Values <a class="anchor" id="nb4_format_llr"></a> 
As mentioned, the LLR values must be formatted correctly before being input to the decoder. The SD-FEC core requires that the LLR values are signed 6-bit wordlengths with 2 fractional bits. This data should also be symmetrically saturated. Where regular saturation of this data type would result in the lower and upper bounds [-8, 7.75], symmetric saturation produces the lower and upper bounds [-7.75,7.75]. 

For the data to be sent using AXI4-Stream, it must be sign-extended to the nearest byte. That means, the sign bit or most significant bit (MSB) of our 6-bit number should be copied and appended to the MSB side of our number twice. Figure 2 illustrates this formatting.

<a class="anchor" id="fig-2"></a>
<center><figure>
<img src='./images/decoder_input.svg' width='350'/>
    <figcaption><b>Figure 2: LLR data format for LDPC decoding.</b></figcaption>
</figure></center>

The process for formatting the LLR data correctly can be broken down into three stages: 
1. Perform symmetric saturation. 
2. Convert to a 6-bit 2's complement.
3. Sign extend to an 8-bit number.

A function is provided below that performs this formatting.

In [ ]:
def format_llr(llr):
    # 1. Perform symmetric saturation
    if llr > 7.75:
        llr = 7.75
    elif llr < -7.75:
        llr = -7.75

    # Represent as integer
    llr = llr * pow(2,2)       # Bit shift to expose 2 fractional values
    llr = int(np.round(llr))   # Truncate remaining fractional values

    # 2. Convert to binary (2's complement)
    if llr >= 0:
        binary = '{0:06b}'.format(llr)
    else:
        binary = '{0:06b}'.format(pow(2,6)+llr)

    # 3. Sign extend to 8-bits
    if binary[0] == '0': 
        binary_extended = '00' + binary
    else:
        binary_extended = '11' + binary

    return int(binary_extended,2)

Using this function, we can loop through our array of LLR values and format each. The LLR values before (blue) and after (red) formatting are plotted.

In [ ]:
llrs_formatted = []
for llr in llrs:
    llrs_formatted.append(format_llr(llr))
    
hf.plot_samples('LLR Values Before and After Formatting',
                [range(80),range(80)],
                [llrs[0:80],hf.interpret_llrs(llrs_formatted)[0:80]])

To further highlight how the LLR values are formatted, the cell below uses a helper function to print out the original LLR values and their formatted counterparts in both decimal and binary form.

In [ ]:
start = 0
n = 10
hf.print_llr_format_table(llrs,start,n)

Our LLR values are now formatted correctly for the SD-FEC core configured as a decoder to interpret.

## 3. Configure Decoder <a class="anchor" id="nb4_configure_decoder"></a>
Configuring the decoder at run-time is the same as configuring the encoder; simply add the desired code parameters to the SD-FEC core. In the cell below, all the available code parameters are added using the helper function *add_all_ldpc_params()*.

In [ ]:
fec = ol.ldpc_decoder.sd_fec
hf.add_all_ldpc_params(fec)

## 4. Setup Buffers <a class="anchor" id="nb4_setup_data_buffers"></a>
As with the encoder, the same four buffers are required in our decoder configuration. A transmit buffer will be utilised to send encoded soft bits to the decoder and a receive buffer will move the decoded block back into the PS for inspection. The other two buffers are used for control and status words which must be sent to and received from the decoder for every data block that is to be decoded. Figure 2 below illustrates the movement of data to and from the SD-FEC core when configured as a decoder.

<a class="anchor" id="fig-2"></a>
<center><figure>
<img src='./images/ctrl_status_dec.svg' width='800'/>
    <figcaption><b>Figure 2: SD-FEC core interfaces when decoding data.</b></figcaption>
</figure></center>

### 4.1. Control <a class="anchor" id="nb4_control"></a>
When the SD-FEC is configured as a decoder, more fields are available for the control interface. These are detailed in table 1 below.

<a class="anchor" id="tab-1"></a>
<center><figure>
    <figcaption><b>Table 1: Control interface for LDPC decoding in Non-5G NR mode.</b></figcaption>
    <br>
    <table style="width:1000">
      <tr>
        <th>Field</th>
        <th>Bits</th>
        <th>Type</th>
        <th>Range</th>
        <th>Description</th>
      <tr style="text-align:center">
        <td>External Block ID</td>
        <td>31:24</td>
        <td>uint8</td>
        <td>0 to 255</td>
        <td>External block identifier to be passed through to status output</td>
      </tr>
      <tr style="text-align:center">
        <td>Max No. Iterations</td>
        <td>23:18</td>
        <td>uint6</td>
        <td>1 to 63</td>
        <td>Maximum number of iterations</td>
      </tr>
      <tr style="text-align:center">
        <td>Terminate on no Change</td>
        <td>17:17</td>
        <td>bit1</td>
        <td>0 or 1</td>
        <td>0: Do not terminate early if there is no change in hard bits for the whole block (information and parity) between iterations<br>
            1: Terminate early if there is no change in hard bits for the whole block (information and parity) between iterations</td>
      </tr>
      <tr style="text-align:center">
        <td>Terminate on Pass</td>
        <td>16:16</td>
        <td>bit1</td>
        <td>0 or 1</td>
        <td>0: Do not terminate early on passing parity check<br>
            1: Terminate early on passing parity check</td>
      </tr>
      <tr style="text-align:center">
        <td>Include Parity Output</td>
        <td>15:15</td>
        <td>bit1</td>
        <td>0 or 1</td>
        <td>0: Output systematic values only<br>
            1: Output systematic values and parity</td>
      </tr>
      <tr style="text-align:center">
        <td>Hard Output</td>
        <td>14:14</td>
        <td>bit1</td>
        <td>0 or 1</td>
        <td>0: Soft output<br>
            1: Hard output</td>
      </tr>
      <tr style="text-align:center">
        <td>-</td>
        <td>13:7</td>
        <td>-</td>
        <td>-</td>
        <td>Reserved</td>
      </tr>
      <tr style="text-align:center">
        <td>Code ID</td>
        <td>6:0</td>
        <td>uint7</td>
        <td>0 to 127</td>
        <td> Code number used to specify which set of LDPC code parameters are to be used on the block</td>
      </tr>
    </table>
</figure></center>

*External Block ID* and *Code ID* were introduced in notebook 2. The *Max No. Iterations* field specifies the maximum number of iterations the decoder performs. Here, a smaller value can reduce latency but might mean that errors have not been corrected as further iterations were required. The reason this is the maximum number is because the following two fields *Terminate on No Change* and *Terminate on Pass* allow the decoder to terminate early if either of these conditions has been met. The *Hard Output* field can be toggled so that the decoder either outputs hard bits or soft bits.

In the code cell below, choose an LDPC code to use when decoding. It is important this matches the code used when encoding. The available codes and their names can be seen in the table output in section 3.

In [ ]:
# Select Code
code_name = 'docsis_medium'
ldpc_params = fec.available_ldpc_params()
code_id = ldpc_params.index(code_name)

# Create control word
ctrl_params = {'id' : 1, 
               'max_iterations' : 63,
               'term_on_no_change' : 1,
               'term_on_pass' : 1, 
               'include_parity_op' : 0,
               'hard_op' : 1,
               'code_id' : code_id}
ctrl_word = hf.create_ctrl_word(ctrl_params,readout='decoder')

# Create control buffer and populate it with control word
ctrl_buffer = allocate(shape=(1,), dtype=np.uint32)
ctrl_buffer[0] = ctrl_word

### 4.2. Status <a class="anchor" id="nb4_status"></a>
The status interface also returns more fields when configured as a decoder. These are detailed in Table 2.

<a class="anchor" id="tab-2"></a>
<center><figure>
    <figcaption><b>Table 2: Status interface for LDPC decoding in Non-5G NR mode.</b></figcaption>
    <br>
    <table style="width:1000">
      <tr>
        <th>Field</th>
        <th>Bits</th>
        <th>Type</th>
        <th>Range</th>
        <th>Description</th>
      <tr style="text-align:center">
        <td>External Block ID</td>
        <td>31:24</td>
        <td>uint8</td>
        <td>0 to 255</td>
        <td>External block identifier supplied through control input</td>
      </tr>
      <tr style="text-align:center">
        <td>Decode Iterations</td>
        <td>23:18</td>
        <td>uint6</td>
        <td>1 to 63</td>
        <td>Number of iterations taken to decode output (either successfully or unsuccessfully)</td>
      </tr>
      <tr style="text-align:center">
        <td>Terminate on no Change</td>
        <td>17:17</td>
        <td>bit1</td>
        <td>0 or 1</td>
        <td>0: Did not terminate early due to no change in hard bits for the whole block (information and parity) between iterations<br>
            1: Terminated early as no change in hard bits for the whole block (information and parity) between iterations</td>
      </tr>
      <tr style="text-align:center">
        <td>Terminate on Pass</td>
        <td>16:16</td>
        <td>bit1</td>
        <td>0 or 1</td>
        <td>0: Did not terminate due to passing parity check<br>
            1: Terminated early due to passing parity check</td>
      </tr>
      <tr style="text-align:center">
        <td>Parity Pass</td>
        <td>15:15</td>
        <td>bit1</td>
        <td>0 or 1</td>
        <td>0: Parity check did not pass <br>
            1: Parity check passed</td>
      </tr>
      <tr style="text-align:center">
        <td>Hard Output</td>
        <td>14:14</td>
        <td>bit1</td>
        <td>0 or 1</td>
        <td>0: Soft output<br>
            1: Hard output</td>
      </tr>
      <tr style="text-align:center">
        <td>Decode Operation</td>
        <td>13:13</td>
        <td>bit1</td>
        <td>0</td>
        <td>Decode operation (fixed value)</td>
      </tr>
      <tr style="text-align:center">
        <td>-</td>
        <td>12:7</td>
        <td>-</td>
        <td>-</td>
        <td>Reserved</td>
      </tr>
      <tr style="text-align:center">
        <td>Code ID</td>
        <td>6:0</td>
        <td>uint7</td>
        <td>0 to 127</td>
        <td>Code number specifying the LDPC code parameters used to decode the block</td>
      </tr>
    </table>
</figure></center>

Again, only the empty buffer is required to be created at this stage. 

In [ ]:
status_buffer = allocate(shape=(1,), dtype=np.uint32)

### 4.3. Transmit and Receive <a class="anchor" id="nb4_transmit_and_receive"></a>
Next, we can create our data buffers. We require one buffer for moving our soft bits into the decoder and another for collecting the decoded hard bits. Let us inspect the parameters for our current code.

In [ ]:
n = fec._code_params.ldpc[code_name]['n']
k = fec._code_params.ldpc[code_name]['k']
p = fec._code_params.ldpc[code_name]['p']

print('Block Length (bits): %s\nInformation Bits: %s\nSub-Matrix Size: %s' % (n, k, p))

As established in section 2 of this notebook, each soft bit requires 8 bits to represent it. For an encoded block, we have *n* soft bits with a wordlength of 8. Our AXI4-Stream data width is also 8 bits, meaning that the length of our transmit buffer should equal *n*, the encoded block size. We have configured the output of the decoder to return hard bits and not to include parity bits, thus the decoder will only output the *k* information bits. Again the AXI4-Stream data width is 8 bits, so our receive buffer length must be *k*/8.

In the code cell below, we create our two data buffers of the correct size and populate the transmit buffer with the *n* formatted LLR values.

In [ ]:
K = int(np.ceil(k/8))

tx_dec_buf = allocate(shape=(n,), dtype=np.uint8)
rx_dec_buf = allocate(shape=(K,), dtype=np.uint8)

j = 0
for i in range(len(tx_dec_buf)):
    tx_dec_buf[i] = llrs_formatted[j]
    j += 1

## 5. Decode Data  <a class="anchor" id="nb4_decode_data"></a>
Having configured the SD-FEC core, formatted our LLR values, and created all of the required buffers, we can now perform LDPC decoding. Running the cell below will send our control word and transmit buffer to the SD-FEC block where the LLR values will be decoded. The decoded data output will be returned in our receive buffer along with a status word in the status buffer. 

In [ ]:
# Assign data and ctrl DMAs to shorter variables
dma_data = ol.ldpc_decoder.axi_dma_data
dma_ctrl = ol.ldpc_decoder.axi_dma_ctrl

# Initiate transfers
dma_ctrl.recvchannel.transfer(status_buffer)
dma_data.recvchannel.transfer(rx_dec_buf)
dma_ctrl.sendchannel.transfer(ctrl_buffer)
dma_data.sendchannel.transfer(tx_dec_buf)

# Wait for transfers to complete
dma_ctrl.sendchannel.wait()
dma_data.sendchannel.wait()
dma_data.recvchannel.wait()
dma_ctrl.recvchannel.wait()

Interpreting and printing the received status word, we can insect the returned fields. Note that if a high SNR was used in the previous notebook, you might see that one or both of the conditions have been met to achieve early termination. After completing this notebook, try varying the SNR level in the previous notebook and completing the steps between there and here again to see how the SNR level impacts on the number of decode iterations performed.

In [ ]:
hf.print_status_reg(status_buffer, 'decoder')

Let us now plot the first 50 bits of the decoded data (blue) and original data generated in notebook 2 (red). Depending on the SNR employed in the previous notebook, you may or may not see differences in the values obtained. You should expect the data to match, mostly, if using SNRs greater than 12 dB. 

In [ ]:
tx_bits = hf.serialise(tx_enc_buf)
rx_bits = hf.serialise(rx_dec_buf)

hf.plot_samples('Tx and Rx',
                [range(50),range(50)],
                [rx_bits[0:50],tx_bits[0:50]])

### 5.1. Bit Error Rate <a class="anchor" id="nb4_ber"></a>

While we can inspect the arrays visually to see if there are errors, it is useful to explore other metrics of assessing the performance of the forward error correction. One such metric is Bit Error Rate or BER. This is a ratio of the number of error bits received over the total number of bits transmitted.

$$
    \mathit{BER} = \frac{N_{error}}{N_{bits}}
$$

In [ ]:
N_error = np.sum(rx_bits != tx_bits)
N_bits = len(tx_bits)
BER = N_error / N_bits
print('Error Bits', N_error)
print('BER:', BER)

While obtaining the BER for one SNR can be useful, it is usually more beneficial to plot a *BER Curve*. This is where BER values are obtained while varying the SNR of the channel and plotting these on a graph. This is a better visualisation of code performance and allows for different codes to be compared more easily. In the next notebook we will combine everything we have touched on thus far to generate BER plots for DOCSIS Short, DOCSIS Medium and DOCSIS Long LDPC codes.

## 6. Conclusion <a class="anchor" id="nb4_conclusion"></a>
This fourth notebook of the series has delved deeper into the details of decoding encoded data using the SD-FEC block on RFSoC. We have learned how to correctly format Log Likelihood Ratios (LLRs) to be input to the decoder, and how to create data buffers of the appropriate size, taking into account the wordlength of the soft LLRs. We also examined the various fields available on the control and status interfaces when the SD-FEC core is operating as a decoder. The decoded data was compared against the original data generated in the second notebook and a performance measure, Bit Error Rate (BER), was introduced. With this knowledge in hand, in the next notebook, we will bring together all the concepts covered in the previous notebooks to generate a series of BER curves for comparing different LDPC codes, which will be a useful tool for assessing and comparing code performance.

---

[⬅️ Previous Notebook](03_fec_channel_simulation.ipynb) || [Next Notebook 🚀](05_fec_bit_error_analysis.ipynb)

Copyright © 2023 Strathclyde Academic Media

---
---